In [ ]:
!pip install comet_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 534.7/534.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 15.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of requests to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: websocke

In [ ]:
#imports
import pandas as pd
import os
from typing import Tuple, Union
from pathlib import Path

import torchaudio
from torchaudio.models.decoder import ctc_decoder
from torchaudio.utils import download_asset
from torch import Tensor
from torch.utils.data import Dataset
import torch.utils.data as data

import os
from comet_ml import Experiment
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

/usr/local/lib/python3.10/dist-packages/torchaudio/models/decoder/_ctc_decoder.py:62: UserWarning: The built-in flashlight integration is deprecated, and will be removed in future release. Please install flashlight-text. https://pypi.org/project/flashlight-text/ For the detail of CTC decoder migration, please see https://github.com/pytorch/audio/issues/3088.
  warnings.warn(


Прикрепляемся к нашему гугл диску, где в корне лежит AudioDiploma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data clening and audio file path allocation

get csv from google drive, rename columns, and remove unused columns

In [ ]:
path = '/content/drive/MyDrive/AudioDiploma/swiss.csv'
df = pd.read_csv(path)


path_bern = '/content/drive/MyDrive/AudioDiploma/clear_bern_test.csv'
df_bern = pd.read_csv(path_bern)

df_bern = df_bern.rename(columns={'path': 'filename', 'swiss translation': 'text'})
df_bern = df_bern.drop(['sentence'], axis=1)


path_bern_train = '/content/drive/MyDrive/AudioDiploma/clear_bern_train36.csv'
df_bern_train = pd.read_csv(path_bern_train)

df_bern_train = df_bern_train.rename(columns={'path': 'filename', 'swiss': 'text'})
df_bern_train = df_bern_train.drop(['sentence'], axis=1)

df_s = [df, df_bern, df_bern_train]
df.shape[0], df_bern.shape[0], df_bern_train.shape[0]

Прописываем пути напротив текста

In [ ]:
def get_path(x):
  '''allocation of corresponding audiofile path'''
  tmp_mark = ''

  if '1235' in x:
    tmp_mark = '1235'
  elif '1240' in x:
    tmp_mark = '1240'
  elif '.flac' in x:
    tmp_mark = 'corpus2test'

  default_path = '/content/drive/MyDrive/AudioDiploma/'
  mark = (str(tmp_mark) + '/') if tmp_mark != '' else ''

  return default_path + mark + x


def get_path_big(x):
  tmp_mark = 'clips'

  default_path = '/content/drive/MyDrive/AudioDiploma/'
  mark = (str(tmp_mark) + '/') if tmp_mark != '' else ''

  return default_path + mark + x

remove punctuation and add file path

In [ ]:
df['text'] = df['text'].str.replace(r'[^\w\s]+', '', regex=True)
df_bern['text'] = df_bern['text'].str.replace(r'[^\w\s]+', '', regex=True)
df_bern_train['text'] = df_bern_train['text'].str.replace(r'[^\w\s]+', '', regex=True)

df['path'] = df.filename.apply(get_path)
df_bern['path'] = df_bern.filename.apply(get_path)
df_bern_train['path'] = df_bern_train.filename.apply(get_path_big)

concat all dfs into one

In [ ]:
df = pd.concat(df_s)
df = df.sample(frac=1).reset_index(drop=True)
df.shape

In [ ]:
df_test = df[34000:]
len(df_test)

4586

### Creation of helper files for Swiss Language model

In [ ]:
# extract all words from swiss text to a vocabulary set
nltk.download('punkt')
vocab = set()
for line in df.text:
    tokens = word_tokenize(line)
    words = [word.lower() for word in tokens if word.isalpha()]
    vocab = vocab.union(set(words))
vocab = sorted(vocab)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
vocab[10505:10510]

['ex', 'exakt', 'exakte', 'exakti', 'exampu']

In [ ]:
# add to lexicon file in format: "word w o r d\n" for each line
with open('lexicon.txt', 'w') as f:
    for word in vocab:
        f.write(word)
        f.write(' ')
        for letter in word:
            f.write(letter)
            f.write(' ')
        #f.write('|')
        f.write('\n')

In [ ]:
# list of tokens
tokens = ['', '|', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '-']

#Text similarity metrics

In [ ]:
def avg_wer(wer_scores, combined_ref_len):
    return float(sum(wer_scores)) / float(combined_ref_len)


def _levenshtein_distance(ref, hyp):
    """Levenshtein distance is a string metric for measuring the difference
    between two sequences. Informally, the levenshtein disctance is defined as
    the minimum number of single-character edits (substitutions, insertions or
    deletions) required to change one word into the other. We can naturally
    extend the edits to word level when calculate levenshtein disctance for
    two sentences.
    """
    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]


def word_errors(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in word-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Levenshtein distance and word number of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)


def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in char-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Levenshtein distance and length of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def wer(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Calculate word error rate (WER). WER compares reference text and
    hypothesis text in word-level. WER is defined as:
    .. math::
        WER = (Sw + Dw + Iw) / Nw
    where
    .. code-block:: text
        Sw is the number of words subsituted,
        Dw is the number of words deleted,
        Iw is the number of words inserted,
        Nw is the number of words in the reference
    We can use levenshtein distance to calculate WER. Please draw an attention
    that empty items will be removed when splitting sentences by delimiter.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Word error rate.
    :rtype: float
    :raises ValueError: If word number of reference is zero.
    """
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case,
                                         delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
    """Calculate charactor error rate (CER). CER compares reference text and
    hypothesis text in char-level. CER is defined as:
    .. math::
        CER = (Sc + Dc + Ic) / Nc
    where
    .. code-block:: text
        Sc is the number of characters substituted,
        Dc is the number of characters deleted,
        Ic is the number of characters inserted
        Nc is the number of characters in the reference
    We can use levenshtein distance to calculate CER. Chinese input should be
    encoded to unicode. Please draw an attention that the leading and tailing
    space characters will be truncated and multiple consecutive space
    characters in a sentence will be replaced by one space character.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Character error rate.
    :rtype: float
    :raises ValueError: If the reference length is zero.
    """
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        char_map_str = """
        ' 0
        <SPACE> 1
        a 2
        b 3
        c 4
        d 5
        e 6
        f 7
        g 8
        h 9
        i 10
        j 11
        k 12
        l 13
        m 14
        n 15
        o 16
        p 17
        q 18
        r 19
        s 20
        t 21
        u 22
        v 23
        w 24
        x 25
        y 26
        z 27
        """
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['<SPACE>']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('<SPACE>', ' ')


# Audio preprocessing

In [ ]:
# return Mel Spectrograms after SpecAugment
train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    torchaudio.transforms.TimeMasking(time_mask_param=100)
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform = TextTransform()

def data_processing(data, data_type="train"):
    '''Function transforms waveforms into spectrograms and
    returns a list of spectrograms,
              list of corresponding labels
              list of input_lengths
              list of label_lengths
    '''
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)


    return spectrograms, labels, input_lengths, label_lengths

/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


## Customized librispeech dataset

In [ ]:
def load_librispeech_item_custom(item) -> Tuple[Tensor, int, str, int, int, int]:
  '''class returns waveforms loaded from audio file'''
  #Load audio
  waveform, sample_rate = torchaudio.load(item.path)
  transcript = item.text

  return (
        waveform,
        0,
        transcript,
        0,
        0,
        0,)

class LIBRISPEECH_CUSTOM(Dataset):

    def __init__(self, df) -> None:
        self._df = df

    def __getitem__(self, n: int) -> Tuple[Tensor, int, str, int, int, int]:
        """Load the n-th sample from the dataset.

        Args:
            n (int): The index of the sample to be loaded

        Returns:
            (Tensor, int, str, int, int, int):
            ``(waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id)``
        """
        fileid = df.iloc[n]
        return load_librispeech_item_custom(fileid)

    def __len__(self) -> int:
        return self._df.shape[0]


# Model and decoders

##Greedy decoder

In [ ]:
def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
  '''function decodes characters from a given probability tensors one by one'''
	arg_maxes = torch.argmax(output, dim=2)
	decodes = []
	targets = []
	for i, args in enumerate(arg_maxes):
		decode = []
		targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
		for j, index in enumerate(args):
			if index != blank_label:
				if collapse_repeated and j != 0 and index == args[j -1]:
					continue
				decode.append(index.item())
		decodes.append(text_transform.int_to_text(decode))
	return decodes, targets

## DNN architecture

In [ ]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time)


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):
    '''Class implements bidirectional RNN with the GRU cell'''
    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    '''class comprises residual CNNs, biRNNs and returns probability tensors'''
    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats)
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x


## Beam decoder

In [ ]:
LM_WEIGHT = 3.23
WORD_SCORE = -0.26
def Beam_Decoder(output):
    '''beam search decoder implementation with swiss language model
      returns a decoded string'''
    beam_search_decoder = ctc_decoder(
        lexicon='lexicon-2.txt',
        tokens=tokens,
        lm='/content/drive/MyDrive/AudioDiploma/LM/swiss_kenlm.bin',
        #lm=None,
        nbest=3,
        beam_size=1500,
        lm_weight=LM_WEIGHT,
        word_score=WORD_SCORE,
    )
    beam_search_result = beam_search_decoder(output)
    beam_search_transcript = " ".join(beam_search_result[0][0].words).strip()
    return beam_search_transcript

# Inference

In [ ]:
def sample_test(model, test_loader):
    '''function returns targets, greedy predictions and beam decoder predictions'''
    model.eval()
    #w = 0
    #c = 0
    with torch.no_grad():
        for i, _data in enumerate(test_loader):
            spectrograms, labels, input_lengths, label_lengths = _data
            spectrograms, labels = spectrograms, labels

#             print(spectrograms.size(), labels.size())
            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            beam_decoded_preds = Beam_Decoder(output)
            # emission = output?
            output = output.transpose(0, 1) # (time, batch, n_class)

            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            #w += wer(decoded_targets[0], beam_decoded_preds)
            #c += cer(decoded_targets[0], beam_decoded_preds)
            #return beam_decoded_preds, decoded_preds[0], decoded_targets[0]
            print('| ---------------------- Result ----------------- |')
            print( 'target: ', decoded_targets[0], '\n', 'decoded wombat: ', decoded_preds[0], '\n', 'decoded beam: ', beam_decoded_preds, '\n')

In [ ]:
import os
os.environ['CUDNN_WARN_NON_CONTIGUOUS'] = '0'

import warnings
warnings.filterwarnings('ignore')

### Model upload from file.pth

In [ ]:
def model_upload(path, learning_rate=5e-4, batch_size=1, epochs=15,
        train_url="train-clean-100", test_url="test-clean"):
    global data
    #train_dataset = LIBRISPEECH_CUSTOM(train_df)
    #test_dataset = LIBRISPEECH_CUSTOM(test_df)

    hparams = {
          "n_cnn_layers": 3,
          "n_rnn_layers": 5,
          "rnn_dim": 512,
          "n_class": 29,
          "n_feats": 128,
          "stride":2,
          "dropout": 0.1,
          "learning_rate": learning_rate,
          "batch_size": batch_size,
          "epochs": epochs
      }

    torch.manual_seed(7)

    model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        )

    # model = nn.DataParallel(model, device_ids=[0])
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

    return model

PATH = '/content/drive/MyDrive/AudioDiploma/train-models/model_v2.pth'
model = model_upload(PATH)

In [ ]:
### Function for telegram decoding

In [ ]:
def decode_telegram_message(audio_path_name):

    d = {'filename': ['name'], 'text':['something was said'], 'path':[audio_path_name]}
    df_unit = pd.DataFrame(data=d)

    sample_dataset = LIBRISPEECH_CUSTOM(df_unit)

    sample_loader = data.DataLoader(dataset=sample_dataset,
                                    batch_size=1,
                                    collate_fn=lambda x: data_processing(x, 'valid'))

    model.eval()
    return sample_test(model, sample_loader)

res = decode_telegram_message('34432.flac')

| ---------------------- Result ----------------- |
target:  dr kanton het im bildigsbereich massnahme troffe drmit persone mit schlechte duetschkenntnis d grundkompetenze choei erreiche 
 decoded wombat:  d kanto he im bildigsbereich massnahme detroffe dami persone mit schlechteduetsketnisu d grundkompetaenze erreichee 
 decoded beam:  dr kanton het im bildigsbereich massnahme troffe drmit persone mit schlechte duetschkenntnis d grundkompetenze erreiche 



### Function for inference for testing test dataset

In [ ]:
def avg_wer(test_df):
    df_unit = pd.DataFrame(data=test_df)

    sample_dataset = LIBRISPEECH_CUSTOM(df_unit)

    sample_loader = data.DataLoader(dataset=sample_dataset,
                                    batch_size=1,
                                    collate_fn=lambda x: data_processing(x, 'valid'))

    model.eval()

    sample_test(model, sample_loader)

In [ ]:
test_df = df.sample(20)
test_df.head(5)

,index,filename,text,path
16880,33973,27358.flac,er cha so sicherstelle dass e aschlussvertrag ...,/content/drive/MyDrive/AudioDiploma/clips/2735...
10475,14568,36490.flac,di mueglechkeit hei juristischi persone nid wi...,/content/drive/MyDrive/AudioDiploma/clips/3649...
9550,32207,37447.flac,mir isch kes angers baernerischs ungernaehme i...,/content/drive/MyDrive/AudioDiploma/clips/3744...
34742,3531,12663.flac,die schaede a gebaeude infrastruktur und kultu...,/content/drive/MyDrive/AudioDiploma/clips/1266...
37949,4913,8714.flac,aber mir muend ufpasse dass nid der unternehme...,/content/drive/MyDrive/AudioDiploma/clips/8714...


In [ ]:
avg_wer(test_df)

| ---------------------- Result ----------------- |
target:  dr kanton het im bildigsbereich massnahme troffe drmit persone mit schlechte duetschkenntnis d grundkompetenze choei erreiche 
 decoded wombat:  d kanto he im bildigsbereich massnahme detroffe dami persone mit schlechteduetsketnisu d grundkompetaenze erreichee 
 decoded beam:  dr kanton het im bildigsbereich massnahme troffe drmit persone mit schlechte duetschkenntnis d grundkompetenze erreiche 

| ---------------------- Result ----------------- |
target:  es geit um ds potentiau fuer stromproduktion us wasserchraft fuer rund sechstausend hushaute 
 decoded wombat:  es geit um ds potenzial fuer stromproduktion uswasserchraft fuerrund sechstausend hushaltige 
 decoded beam:  es geit um ds potentiau fuer stromproduktion us wasserchraft fuer rund sechstausend hushaute 

| ---------------------- Result ----------------- |
target:  mir stelle ues dere diskussion und werde vorschlaeg praesentiere aber no nid huet 
 decoded wombat: 